In [165]:
! pip install mediapipe opencv-python


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import cv2
import mediapipe as mp

In [3]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [170]:

import cv2
import mediapipe as mp
import numpy as np

counter = 0 
stage = None

cap = cv2.VideoCapture(0)
with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        results = pose.process(image)

        mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            tmp = results.pose_landmarks.landmark
            wrist = [tmp[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].x, tmp[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].y]
            elbow = [tmp[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x, tmp[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].y]
            shoulder = [tmp[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x, tmp[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            angle = cal(shoulder, elbow, wrist)
            cv2.putText(image,str(angle),
                        tuple(np.multiply(elbow, [width, height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1

        except:
            pass

        
        cv2.rectangle(image, (0,0), (250,73), (245,117,16), -1)
        
      
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


1
2
3
4
5
6


In [167]:
def cal(a,b,c):
    a = np.array(a) 
    b = np.array(b) 
    c = np.array(c) 
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 